In [1]:
import boto3

In [33]:
import requests

In [38]:
r = requests.get(url)

In [1]:
from tweets import get_tweets
from util import cleanUpTweet, getTextSubjectivity, getTextAnalysis, getTextPolarity
import datetime
import pandas as pd
from sqlalchemy import text
from sqlalchemy import create_engine
import numpy as np

In [2]:
db = create_engine('bigquery://', credentials_path="BIGQUERY_CREDENTIAL.json")

In [4]:
q = "select * from twittersentiment-339717.tweets.tweets_labeled"

In [5]:
df = pd.read_sql(q, con = db)

In [19]:
df.head()

,tweet_date,pos,neg,neu,query,pos_percentage
0,2022-01-27,8,4,3,biden,53.333333
1,2022-01-27,9,3,3,trump,60.000000
2,2022-01-23,6,6,3,biden,40.000000
3,2022-01-24,7,5,3,biden,46.666667
4,2022-01-25,4,5,6,biden,26.666667


In [18]:
df['pos_percentage'] = (df['pos']/15) * 100

In [20]:
df_temp = df.copy()

In [28]:
df_temp.head()

,tweet_date,pos,neg,neu,query,pos_percentage
0,2022-01-27,8,4,3,biden,53.333333
1,2022-01-27,9,3,3,trump,60.000000
2,2022-01-23,6,6,3,biden,40.000000
3,2022-01-24,7,5,3,biden,46.666667
4,2022-01-25,4,5,6,biden,26.666667


In [29]:
df_temp.head()
df_temp = df_temp.drop(['pos_percentage'], axis = 1)
df_temp.head()

,tweet_date,pos,neg,neu,query
0,2022-01-27,8,4,3,biden
1,2022-01-27,9,3,3,trump
2,2022-01-23,6,6,3,biden
3,2022-01-24,7,5,3,biden
4,2022-01-25,4,5,6,biden


In [37]:
df_temp.head()

,tweet_date,pos,neg,neu,query
0,2022-01-27,8,4,3,biden
1,2022-01-27,9,3,3,trump
2,2022-01-23,6,6,3,biden
3,2022-01-24,7,5,3,biden
4,2022-01-25,4,5,6,biden


In [46]:
df_temp = df_temp.rename(columns={'pos': 'label_pos', 'neg': 'label_neg', 'neu': 'label_neu'})

In [55]:
df_temp.head()

,tweet_date,label_pos,label_neg,label_neu,query
0,2022-01-27,8,4,3,biden
1,2022-01-27,9,3,3,trump
2,2022-01-23,6,6,3,biden
3,2022-01-24,7,5,3,biden
4,2022-01-25,4,5,6,biden


In [63]:
fig = px.bar(df[df['query'] == "biden"], x="tweet_date", y=["pos", "neg", "neu"], title="Biden Subjectivity by Date", labels = {'tweet_date': 'Date',
                      'value': 'Subjectivity Count', 'variable': ' Subjectivity'})
fig.show()

In [42]:
df1 = pd.DataFrame({'id': [1,2], 'A_1': ['a', 'b'],
                  'A_2': ['aa', 'bb'], 'A_3': ['aaa', 'bbb'],
                  'A_person': ['Mike', 'Amy']})

In [43]:
df1

,id,A_1,A_2,A_3,A_person
0,1,a,aa,aaa,Mike
1,2,b,bb,bbb,Amy


In [56]:
t = pd.wide_to_long(df_temp, stubnames='label', i = ['tweet_date', 'query'], j = 'Subjectivity').reset_index()

In [57]:
t.head()

,tweet_date,query,Subjectivity,label_neu,label_pos,label_neg,label


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   tweet_date  12 non-null     datetime64[ns]
 1   pos         12 non-null     int64         
 2   neg         12 non-null     int64         
 3   neu         12 non-null     int64         
 4   query       12 non-null     object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 608.0+ bytes


In [7]:
import plotly.express as px

In [14]:
df_temp = df[df['query'] == "biden"].sort_values(by=['tweet_date'])

In [16]:
df_temp1 = df.sort_values(by=['tweet_date'])

In [15]:
fig = px.line(df_temp, x="tweet_date", y="pos")
fig.show()

In [17]:
fig = px.line(df_temp1, x="tweet_date", y="pos", color = 'query')
fig.show()

In [23]:
def tweets_to_database(topic, date):
    tweets, dts, tweet_ids = get_tweets(topic, 15, date)
    tweets = list(map(cleanUpTweet, tweets))
    db = create_engine('bigquery://', credentials_path="BIGQUERY_CREDENTIAL.json")
    headers = ['twitter_id', 'tweet_date', 'tweet', 'query']
    queries = [topic]*len(tweets)
    dates = [date] * len(tweets)
    lists = [tweet_ids, dates, tweets, queries]
    df = pd.DataFrame(lists).transpose()
    df.columns = headers
    df.to_sql('twittersentiment-339717.tweets.tweets', db, if_exists='append', index=False)
    df = pd.DataFrame(data=[tweet for tweet in tweets], columns=['Tweet'])
    df['Tweet'] = df['Tweet'].apply(cleanUpTweet)
    df['Date'] = date
    df['Subjectivity'] = df['Tweet'].apply(getTextSubjectivity)
    df['Polarity'] = df['Tweet'].apply(getTextPolarity)
    df = df.drop(df[df['Tweet'] == ''].index)
    df['Score'] = df['Polarity'].apply(getTextAnalysis)
    labels = df.groupby('Score').count().index.values
    values = df.groupby('Score').size().values
    if "Negative" not in labels:
        values = np.insert(values, 0, 0)
    if "Neutral" not in labels:
        values = np.insert(values, 1, 0)
    if "Positive" not in labels:
        values = np.append(values, 0)
    neg = int(values[0])
    neu = int(values[1])
    pos = int(values[2])
    lists = [[date], [pos], [neg], [neu], [topic]]
    df_labeled = pd.DataFrame(lists).transpose()
    headers = ['tweet_date', 'pos', 'neg', 'neu', 'query']
    df_labeled.columns = headers
    df_labeled.to_sql('twittersentiment-339717.tweets.tweets_labeled', db, if_exists='append', index=False)
    return True

In [30]:
dates = []
for i in range(0, 10):
    dates.append(str(datetime.date.today() - datetime.timedelta(days=i)))
dates = dates[::-1]

In [31]:
dates

['2022-01-20',
 '2022-01-21',
 '2022-01-22',
 '2022-01-23',
 '2022-01-24',
 '2022-01-25',
 '2022-01-26',
 '2022-01-27',
 '2022-01-28',
 '2022-01-29']

In [ ]:
date = str(datetime.date.today())

In [32]:
tweets_to_database("biden")

TypeError: tweets_to_database() missing 1 required positional argument: 'date'

In [2]:
topic = "biden"
date = str(datetime.date.today())

In [63]:
tweets, dts, tweet_ids = get_tweets(topic, 15, date)

In [66]:
tweets = list(map(cleanUpTweet, tweets))

In [67]:
tweets

['Governor DeSantis is taking credit for  over $80 million for infrastructure in Florida\n\nBut Florida’s entire GOP congressional…',
 'Welcome to the Biden family, Willow! ',
 'This record-setting enrollment period was a result of the hard work of  and Democrats in Congress. By passing the Am…',
 "We're talking about:\n\n• The Civil Rights Movement (as a fraud)\n• CRT in schools\n• The Pandemic\n• Biden's Presidency and mor…",
 'The House majority runs through Central Florida. If you help me flip this +8 Biden district blue, you will never have to worr…',
 'The  administration took nearly $7 billion allocated for medical facilities during the pandemic and used it to pu…',
 'Reagan: I’m going to nominate a woman to the Court. \n\nTrump: I’m going to nominate a woman to the Court. \n\nBiden: I’m going…',
 'Donald Trump and Ronald Reagan both promised to nominate women to the Supreme Court, then followed through. So Joe Biden’s…',
 'Pennsylvania is proud to host President Biden today — a

In [5]:
import pandas as pd
from sqlalchemy import text
from sqlalchemy import create_engine

In [6]:
from sqlalchemy import text
from sqlalchemy import create_engine
from google.cloud import bigquery
from google.oauth2 import service_account


In [7]:
credentials = service_account.Credentials.from_service_account_file('BIGQUERY_CREDENTIAL.json')

In [8]:
client = bigquery.Client(credentials=credentials)

In [ ]:
query = """
INSERT INTO bigquery-public-data.tweets.tweets (twitter_id, tweet_date, tweet, query)
VALUES ()

"""

In [9]:
db = create_engine('bigquery://', credentials_path="BIGQUERY_CREDENTIAL.json")

In [46]:
headers = ['twitter_id', 'tweet_date', 'tweet', 'query']
queries = [topic]*len(tweets)
dates = [date] * len(tweets)

In [55]:
lists = [tweet_ids, dates, tweets, queries]

In [56]:
lists = [tweet_ids, dates, tweets, queries]
df = pd.DataFrame(lists).transpose()
df.columns = headers

In [57]:
df.head()

,twitter_id,tweet_date,tweet,query
0,1487213901087481857,2022-01-29,RT @lindyli: Governor DeSantis is taking credi...,biden
1,1487213900659675139,2022-01-29,"RT @POTUS: Welcome to the Biden family, Willow...",biden
2,1487213900458254338,2022-01-29,RT @BarackObama: This record-setting enrollmen...,biden
3,1487213900214976513,2022-01-29,RT @HotepJesus: We're talking about:\n\n• The ...,biden
4,1487213899816439809,2022-01-29,RT @gellered: The House majority runs through ...,biden


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   twitter_id  15 non-null     object
 1   tweet_date  15 non-null     object
 2   tweet       15 non-null     object
 3   query       15 non-null     object
dtypes: object(4)
memory usage: 608.0+ bytes


In [59]:
df.to_sql('twittersentiment-339717.tweets.tweets', db, if_exists='append', index=False)